# Optimization of Lennard-Jones potential for low density limit

### Loss function based on the second virial coefficient (function of pressure, temperature and LJ parameters)

$$ L\left(\sigma, \epsilon, P, T\right) = V\sum_{P, T}\left(\frac{P}{k_BT}\right)^3
\left[B_{t}\left(T\right) - B_{m}\left(\sigma,\epsilon,T\right)\right]^2 $$

### Alternatively (with higher accuracy over a wider range of densities), loss function based on the density (function of pressure, temperature and LJ parameters)

$$ L\left(\sigma, \epsilon, P, T\right) = \frac{V}{4}\sum_{P, T}
\frac{\left[\rho_{t}\left(T\right) - \rho_{m}\left(\sigma,\epsilon,T\right)\right]^2}{\frac{\rho_{T}\left(T\right)}{1+2B_{t}\rho_{t}} + \frac{\rho_{m}\left(\sigma,\epsilon,T\right)}{1+2B_{m}\rho_{m}}} $$

**with number density ($mol/m^3$) determined from equation of state as**

$$ \rho = \frac{-1 + \sqrt{1+\frac{4BP}{RT}}}{2B} $$

### Loss function based on diffusion coefficient (distance between gaussian distributions)

$$ L\left(\sigma, \epsilon, P, T\right) = \ln\frac{D_{target}+D_{model}}{2 \sqrt{D_{target}D_{model}}} $$

In [3]:
import numpy as np
from scipy.optimize import minimize

## Second virial coefficient as a function of temperature and LJ parameters

In [4]:
def b2(T, sig=1.0, eps=1.0, verbose=False):
    """Returns second virial coefficient of Lennard-Jones fluid
    
    Parameters
    ----------
    T : float
        Absolute temperature (K)
    sig : float
        Size parameter (Angstrom)
    eps : float
        Energy parameter (K)
        
    Returns
    -------
    b2 : float
        second virial coefficient (m^3/mol)
    """
    
    # Reduced temperature
    #Tr = T*8.314472/(eps*1000.0)
    Tr = T/eps
    
    # Check if the reduced temperature is safe range
    if verbose and (Tr < 0.3 or Tr > 10000.0):
        print(f'Temperature {Tr} K is not in range 298-3000 K.')
        
    # Reduced virial coefficient
    Br  = 1.9863*Tr**(-0.25) - 1.5985*Tr**(-0.5) - 6.7636*Tr**(-1.5)
    Br += 8.0952*Tr**(-2.0) - 4.8797*Tr**(-2.5) + 0.89417*Tr**(-3.5)
    Br -= 0.25573*Tr**(-4.0) - 0.015386*Tr**(-5.0)
    
    Na = 6.0221409e23
    b0 = 2*np.pi*Na*(sig*1e-10)**3/3
    
    B2 = Br*b0
    
    return B2

In [16]:
def density(P, T, b2):
    
    R = 8.314472
    rho = (-1 + np.sqrt(1 + 4*P*b2/(R*T))) / (2*b2)
    
    return rho

## Reduced collision integrals for LJ potential

In [5]:
class Omega:
    """Collision integrals for Lennard-Jones potential"""
    
    def __init__(self, coeffs):
        if isinstance(coeffs, dict):
            self.coeffs = coeffs
        elif isinstance(coeffs, np.ndarray):
            self.coeffs = self.make_dict(coeffs)
        else:
            raise TypeError
            
    def make_dict(self, coeffs_array):
        """Creates a dictionary from supplied coefficient ndarray.
        
        Works for a particular structure of ndarray - should be modified
        if other format is used.
        """
        
        ls_tuples = [(1, i) for i in range(1, 8)] + [(2, i) for i in range(2, 7)] + [(3,3), (3,4), (3,5), (4,4)]
        b_list = [1, 3, 5, 8, 10, 6]
        c_list = [2, 4, 7, 9, 11, 12]
        b_fac = np.array([1., 1., 10., 10., 100., 1000.])
        c_fac = np.array([1., 10., 10., 100., 1000., 10000.])

        coeffs = {}
        for i in range(16):
            ls = ls_tuples[i]
            coeffs[ls] = {}
            coeffs[ls]['a'] = coeffs_array[0,i]
            coeffs[ls]['b'] = coeffs_array[b_list, i]/b_fac
            coeffs[ls]['c'] = coeffs_array[c_list, i]/c_fac

        return coeffs
        
    def get(self, Tr, l=1, s=1):
        """Returns collision integral for a given temperature"""
        coeff = self.coeffs[(l, s)]
        b = coeff['b']
        c = coeff['c']
        omega = coeff['a']
        omega += sum([b[k-1]*Tr**(-k) + c[k-1]*np.log(Tr)**k for k in range(1, 7)])
    
        return omega

In [ ]:
def diff_coeff()

In [6]:
# read data from file
cc = np.loadtxt('../data/raw/coeffs_num.txt')

In [7]:
omg = Omega(cc)

In [8]:
omg.get(1.0, 2, 6)

1.1127095589290001

In [18]:
def diff_correction(Tr):
    
    o_11 = omg.get(Tr, 1, 1)
    o_12 = omg.get(Tr, 1, 2)
    o_13 = omg.get(Tr, 1, 3)
    o_22 = omg.get(Tr, 2, 2)
    
    A = o_22/o_11
    B = (5*o_12 - 4*o_13)/o_11
    C = o_12/o_11
    
    fd = 1 - (6*C - 5)**2 / (55 - 12*B + 16*A)
    fd /= 1.0
    
    return fd

## Target data

* Second virial coefficient over a range of temperatures (i.e., low density equation of state)
* Low-density viscosities

In [14]:
# (P, T) conditions
T_list = np.linspace(300, 2500, 23)
PT_list = [(1., T) for T in T_list]
#PT_list

## Optimization

* Optimize over a range of temperatures (and pressures)

In [ ]:
def loss_rho(params, PT_list, rho_target):
    
    assert len(T_list) == len(P_list), 'Lengths of P and T lists do not match'
    
    # ignoring prefactor V/4
    loss = 0.0
    for P, T, b2_m, b2_t in zip(P_list, T_list, b2_model, b2_target):
        loss += (P/T)**3 * (b2_m - b2_t)**2
    
    return loss

In [12]:
def loss_b2(params, PT_list, b2_target):
    
    assert len(T_list) == len(P_list), 'Lengths of P and T lists do not match'
    
    # list of model B2 for a given list of temperatures
    b2_model = [b2(T, params[0], params[1]) for P, T in PT_list]
    
    # ignoring prefactor V/(k_B)^3
    # If P is only one (P->0), it can be included in the prefactor
    loss = 0.0
    for P, T, b2_m, b2_t in zip(P_list, T_list, b2_model, b2_target):
        loss += (P/T)**3 * (b2_m - b2_t)**2

    return loss

In [13]:
# initial parameters (sigma (A), epsilon (K))
params_in = np.array([2.92, 393.7])

In [15]:
result = minimize(loss_b2, params_in, args=(PT_list, b2_target))
result

NameError: name 'b2_target' is not defined